# Optimization of the best model

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm
import matplotlib.pyplot as plt
from Helpers.data import Data
from Helpers.models import Models
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow_addons.metrics import F1Score

## Build Model

In [ ]:
def residual_block(x, filters=128, stride=2):
    # Ejemplo simplificado: conv->bn->relu->conv->add->bn->relu->maxpool
    shortcut = x
    
    out = keras.layers.Conv1D(filters, stride, padding='same')(x)
    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.ReLU()(out)
    
    out = keras.layers.Conv1D(filters, stride, padding='same')(out)
    out = keras.layers.Add()([shortcut, out])
    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.ReLU()(out)
    out = keras.layers.MaxPool1D(pool_size=2, strides=2)(out)
    return out

def build_model(hp):
    """
    Crea un modelo Keras “similar” a tu customResNet,
    con hiperparámetros ajustables (HP).
    """

    inputs = keras.Input(shape=(15,114))

    # 1) Conv inicial (fijo en este ejemplo)
    x = keras.layers.Conv1D(128, 2)(inputs)

    # 2) Tres bloques residuales (similar a customResNet).
    #    Podríamos incorporar hiperparámetros aquí también (e.g. # de bloques).
    for i in range(3):
        x = residual_block(x, filters=128, stride=2)

    # 3) Flatten
    x = keras.layers.Flatten()(x)

    # 4) Añadimos capas densas con hiperparámetros
    # p.ej. n_capas_densas y dropout:
    n_capas = hp.Int("n_capas_densas", min_value=1, max_value=3)
    for i in range(n_capas):
        units = hp.Choice(f"units_dense_{i}", [64,128,256])
        x = keras.layers.Dense(units, activation='relu')(x)
        # dropout
        rate = hp.Float(f"dropout_{i}", min_value=0.0, max_value=0.8, step=0.1)
        x = keras.layers.Dropout(rate)(x)

    # 5) Capa de salida (número de clases a tu elección)
    num_classes = 122
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs, name="MyResnet1D")

    # 6) Definir la tasa de aprendizaje como hiperparámetro
    lr = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=[F1Score(num_classes=num_classes, average='macro')]
    )
    return model


## Optimization search

In [ ]:
def optimization_search(data_obj, dir_path, subjects_array):
    f1_results = []  # almacenar F1 final por sujeto

    for sub in tqdm(subjects_array, desc="Procesando sujetos", unit="sujeto"):
        train_data, test_data = data_obj.LeaveOneOutExp1(sub)

        if len(train_data) == 0 or len(test_data) == 0:
            print(f"[ADVERTENCIA] Sujeto {sub}: sin datos, se omite.")
            continue

        X_train, y_train = data_obj.SplitXandY(train_data)
        X_test, y_test   = data_obj.SplitXandY(test_data)

        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
        )

        (y_train_enc, y_val_enc, y_test_enc), le = data_obj.EncodeLabels([y_train, y_val, y_test])
        input_shape = (X_train.shape[1], X_train.shape[2])
        n_classes   = y_train_enc.shape[1]  # asume one-hot => (samples, n_classes)
        class_names = le.classes_
        labels = np.arange(len(class_names))
        y_true = y_test_enc.argmax(axis=1)

        # Definir tuner con la métrica 'val_f1' (expuesta por la custom metric)
        tuner = kt.RandomSearch(
            build_model,                 # la función que construye el modelo
            objective=kt.Objective('val_f1_score', direction='max'),          # métrica a optimizar
            max_trials=10,
            executions_per_trial=1,
            directory=dir_path,
            project_name=f"resnet_opt_{sub}"
        )

        tuner.search(
            X_train, y_train_enc,
            validation_data=(X_val, y_val_enc),
            epochs=25,
            batch_size=32,
            verbose=1
        )

        best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

        best_model = tuner.hypermodel.build(best_hp)
        best_model.fit(
            X_train, y_train_enc,
            validation_data=(X_val, y_val_enc),
            epochs=25, 
            batch_size=32,
            verbose=1
        )

        # Evaluar en el test con 'model.evaluate' => [loss, f1]
        results = best_model.evaluate(X_test, y_test_enc, verbose=0)
        # Por definicion: results[0] = loss, results[1] = f1 (la segunda métrica)
        test_loss, test_f1 = results[0], results[1]

        f1_results.append(test_f1)

        # Guardar si deseas
        best_model.save(f"./{dir_path}/resnet_opt_{sub}/best_model.h5")

        y_pred_probs = best_model.predict(X_test, batch_size=32, verbose=0)
        y_pred = y_pred_probs.argmax(axis=1)
        report = classification_report(y_true, y_pred, labels=labels, target_names=class_names, zero_division=0, output_dict=True)
        f1_acc = report['macro avg']['f1-score']

        with open(f"./{dir_path}/resnet_opt_{sub}/outputs.txt", "w") as f:
            print(best_hp.values, file=f)
            print(f"Sujeto {sub}: Test F1 = {test_f1:.3f}, Loss = {test_loss:.3f}", file=f)
            print(f"Calculado con Classification_report: F1-score = {f1_acc:.3f}", file=f)



    # Al final, calculas promedio y desviación estándar de F1:
    return np.array(f1_results)

In [ ]:
df = pd.read_csv("./Data/Dataset.csv")
data_obj = Data(df=df)
subjects_array = [1,2,3,4,5,6,7,8,9,10]

In [ ]:
f1_results = optimization_search(data_obj=data_obj, dir_path=f"./Op results", subjects_array=subjects_array)
subject_scores = {f"Sub_{i+1}": f1 for i, f1 in enumerate(f1_results)}